In [1]:
import pandas as pd
import numpy as np
import sys
import os

import time, enum, math
import pylab as plt
import random
import warnings
warnings.filterwarnings('ignore')

from state import State
from Agent import MyAgent
from NetworkInformationDiffusionModel import NetworkInformationDiffusionModel
from Visualization import Visualization
from Data import Data
import seaborn as sns

import networkx as nx

import warnings
warnings.filterwarnings('ignore')

Loading BokehJS ...

In [2]:
def compute_share_prob(potential_users):
    
    user_id = None
    
    sharing_prob = potential_users['user_activity'] * potential_users['privacy_preference']
    
    # 50% of the times make a random selection of author
    # if(random.random() > 0.50  or user_id == None):
    rnd = random.random()
    
    if(rnd > 0.50):
        user_id = int(potential_users.sample(1)['id'].values[0])
        
    elif(len(sharing_prob) > 0):
        user_id = int(sharing_prob.idxmax())
        
    return user_id
    
def pick_an_author(data, post):
    
    issue_id = post[0]
    post_stance = post[1]
    issue_mentioned = 'issue_' + str(int(issue_id))
    user_id = None
    
    if data[data[issue_mentioned] <= 0].shape[0] == 0 or data[data[issue_mentioned] >= 0].shape[0] == 0:
        return None
    
    agent_inclination = data[issue_mentioned]
    if(post_stance >= 0):
        potential_users = data[data[issue_mentioned] >= 0]
        selected_author = compute_share_prob(potential_users)
    
    else:
        
        potential_users = data[data[issue_mentioned] < 0]
        selected_author = compute_share_prob(potential_users)
    
    return selected_author

In [3]:
def compare_data(df, data, attr):
    
    for i in range(df.shape[0]):
        if(df.iloc[i][attr] != data.iloc[i][attr]):
            print("old value ", df.iloc[i][attr], " \t New value", data.iloc[i][attr])

def update_user_activity(data, G, user, agg_sanct):
    
    old_user_activity = data.copy().iloc[user]['user_activity']
    
    ## NEED TO CHANGE THIS FORMULA TO BE SOMETHING MORE MEANINGFUL
    new_user_activity = old_user_activity + (agg_sanct * 0.1)
    
    #Update the user activity with the new value (a bounded value between 0 and 1)
    new_user_activity = max(min(1, new_user_activity), 0)
    #data.at[user, 'user_activity'] = new_user_activity
    
    if(old_user_activity == new_user_activity and agg_sanct > 1e-14 and old_user_activity != 1):
        print(old_user_activity)
        print(agg_sanct)
        sys.exit()
    
    data['user_activity'].iloc[user] = new_user_activity
    G.nodes[user]['user_activity'] = new_user_activity
    
    return data, G
    
def attitude_shift(sanction_score, att_diff, author_inclination, reciever_inclination):
    
    attitude_shft = 0.1 * sanction_score * 1/(att_diff+1)
    if(author_inclination >= reciever_inclination):
        new_user_incl = author_inclination - attitude_shft
    else:
        new_user_incl = author_inclination + attitude_shft
        
    return new_user_incl


def update_user_satisfaction(data, G, user, agg_sanct):
    
    user_data = data.iloc[user]
    
    old_user_satisfaction = user_data['user_satisfaction']
    
    ## NEED TO CHANGE THIS FORMULA TO BE SOMETHING MORE MEANINGFUL
    new_user_satisfaction = old_user_satisfaction + (agg_sanct * 0.1)
    
    data['user_satisfaction'].iloc[user] = new_user_satisfaction
    G.nodes[user]['user_satisfaction'] = new_user_satisfaction
    
    return data, G

def clamp(num, minn, maxx):
    return minn if num < minn else maxx if num > maxx else num
    
def update_user_preferences(sG, G, data, issue_id, lower_attd_th, upper_attd_th):
    
    labels = nx.get_edge_attributes(sG,'weight')
    outgoing = list(set([x[0] for x in labels.keys()]))
    incoming = list(set([x[1] for x in labels.keys()]))
    updated_data = data.copy()
    
    for n in incoming:
        lab = [labels[x] for x in labels if x[1] == n]
        agg_sanct = sum(lab)/len(lab)
        
        # print(agg_sanct)
        
        updated_data, G = update_user_activity(data.copy(), G, n, agg_sanct)
        updated_data, G = update_user_satisfaction(updated_data, G, n, agg_sanct)
        
    for node in incoming:
        
        edges = sG.in_edges(node, data=True)
        gama = 0
        temp_df = pd.DataFrame(columns = ['author', 'reciever', 'att_diff', 'sanction_scores'])
        
        for x in edges:
            reciever = x[0]
            author = x[1]
            sanction_score = x[2]['weight']

            reciever_inclination = updated_data[updated_data['id'] == reciever]['issue_' + str(issue_id)].values[0]
            author_inclination = updated_data[updated_data['id'] == author]['issue_' + str(issue_id)].values[0]
            att_diff =  abs(reciever_inclination - author_inclination)
            
            temp_dict = {'author':author_inclination, 'reciever':reciever_inclination, 'att_diff':att_diff, 'sanction_scores':sanction_score}
            temp_df = temp_df.append(temp_dict, ignore_index=True)
            
        temp_df['adj_att_diff'] = temp_df['att_diff'] + 1
        
        temp_df['att_shift'] = temp_df['sanction_scores'] * 0.1/temp_df['adj_att_diff']
        
        postive_shift = temp_df[temp_df['att_diff'] < lower_attd_th]
        negative_shift = temp_df[temp_df['att_diff'] > upper_attd_th]
        
        if(postive_shift.shape[0] > 0 and negative_shift.shape[0] > 0):
            att_shift = postive_shift['att_shift'].mean() - negative_shift['att_shift'].mean()
        elif(postive_shift.shape[0] == 0 and negative_shift.shape[0] > 0):
            att_shift = (-1) * negative_shift['att_shift'].mean()
        elif(negative_shift.shape[0] == 0 and postive_shift.shape[0] > 0):
            att_shift = postive_shift['att_shift'].mean()
        else:
            att_shift = 0
            
        att_shift = clamp(att_shift, -0.20, 0.20)

        new_user_incl = author_inclination + att_shift
        new_user_incl = clamp(new_user_incl, -1, 1)
        
        if(math.isnan(new_user_incl)):
            print("here")
            print("author_inclination ", author_inclination)
            print("att_shift ", att_shift)
            print("new_user_incl ", new_user_incl)
            print("postive_shift \n", postive_shift)
            print("negative_shift \n", negative_shift)
            print("DF \n", temp_df)
            sys.exit(0)
        
        updated_data['issue_' + str(issue_id)].iloc[author] = new_user_incl
        G.nodes[author]['issue_' + str(issue_id)] = new_user_incl
    
    return updated_data, G

def update_pol_pol_in_graph(data, G):
    
    for n in G:
        G.nodes[n]['pol_inclination'] = data.iloc[n]['pol_inclination']
        
    return G


def run_simulation(post, G, steps, seed):

    model = NetworkInformationDiffusionModel(post, G, se_flag, se_threshold, issue_weights, seed)
    for i in range(steps):

        model.reset_randomizer(seed)
        model.step(i)
        #agent_state = model.datacollector.get_agent_vars_dataframe()
        #X = pd.pivot_table(agent_state.reset_index(), index='Step', columns='State', aggfunc=np.size, fill_value=0)  
        
    #print(model.datacollector.get_agent_vars_dataframe())
    agent_state = model.datacollector.get_agent_vars_dataframe()
    
    states = [int(i.state) for i in model.grid.get_all_cell_contents()]
    agents = model.agents
    
    return model, states, agents, model.G, model.G_share, agent_state

def start_simulation(c_i, c_k, data, G, post_conf, n_issues, lower_attd_th, upper_attd_th, seed, Data_obj):
    
    steps=50
    SimModel, states, agents, AgentGraphs, SancGraphs, AgentStates, runtime = {}, [], [], [], [], [], []
    network_homophily = []
    net_homophily = []
    polarization, polarization2 = [], []
    user_satisfaction = []
    user_activity = []
    users_activities = []
    polarity = []
    avg_polarity = []
    agent_pol_inclination = []
    i = 0
    sharing_details = []
    updated_data = None
    print(c_k, "\t", c_i, "\t")   
    
    for j, post in post_conf.iterrows():

        i+=1
        print(i, end="\r")
        
        pc = (post['issue'], post['stance'])
        #print("post ", pc)

        st=time.time()
        author_id = pick_an_author(data, pc)
        #author_id = int(post['author_id'])
        
        if(author_id == None):
            continue

        post = dict()
        post['author'] = author_id
        post['issue'] = int(pc[0])
        post['stance'] = pc[1]

        SimModel[j], state, agent, G_agents, G_sanctions, agent_state = run_simulation(post, G, steps, seed)
        
        et = time.time()
        rt = round(et-st, 6)

        states.append(state) 
        agents.append(agent)
        AgentGraphs.append(G_agents)
        SancGraphs.append(G_sanctions)
        AgentStates.append(agent_state)
        runtime.append(rt)

        author = author_id
        #received_agents_count = sum([1 for x in state if x == 1])
        not_received_agents_count = sum([1 for x in state if x == 2])
        spreader_agents_count = sum([1 for x in state if x == 3])
        disinterested_agents_counts = sum([1 for x in state if x == 4])
        received_agents_count = spreader_agents_count + disinterested_agents_counts

        #print(state)
        sharing_details.append([pc[0], round(pc[1], 6), author_id, received_agents_count, not_received_agents_count, spreader_agents_count, disinterested_agents_counts])
        #print(sharing_details)

#         Visualization().plot_sim_network(G_agents, state)
#         Visualization().plot_sanction_graph(G_sanctions)
        # sys.exit()
        
        # Update user preferences based on sanctions received from other agents
        data_updated, G_agents = update_user_preferences(G_sanctions, G_agents, data.copy(), post['issue'], lower_attd_th, upper_attd_th)
        
        updated_data = data_updated.copy()
        
        # print("Comparison ", data['user_activity'].compare(updated_data['user_activity']))
        data = data_updated
        
#         print(updated_data.shape, updated_data.columns)
#         print(n_issues)
        
#         issues = ['issue_' + str(x) for x in range(n_issues)]
        
#         kx = updated_data[issues].mean(axis = 1)
#         temp_df = updated_data[issues]
#         print(temp_df.shape, temp_df.columns)
#         print(temp_df.head())
#         print(kx)
        
#         sys.exit()
        
        pol_inclination = Data_obj.get_agent_pol_inclinations(updated_data, n_issues)
        
        if(updated_data['pol_inclination'].isna().sum() > 0):
            #print(pol_inclination)
            
            print("updated_data ", updated_data)
            sys.exit()
            
        updated_data['pol_inclination'] = pol_inclination
        
        agent_pol_inclination.append(pol_inclination)
        
        G_agents = update_pol_pol_in_graph(updated_data, G_agents)
        
        net_user_satisfaction = updated_data['user_satisfaction'].mean()
        user_satisfaction.append(net_user_satisfaction)
        
        mean_user_activity = updated_data['user_activity'].mean()
        users_activities.append(updated_data['user_activity'])
        user_activity.append(mean_user_activity)
        
        pol = round(math.sqrt(sum([x*x for x in updated_data['pol_inclination']])/updated_data.shape[0]), 6)
        
        if(math.isnan(pol)):
            
            print(updated_data['pol_inclination'])
            sys.exit()
        
        avg_pol = round(updated_data['pol_inclination'].mean(), 6)
        
        temp_G = G.copy()
        
        node_attr = updated_data.set_index('id').to_dict('index')
        nx.set_node_attributes(temp_G, node_attr)
         
        for n in temp_G.nodes:
            if(node_attr[n]['pol_inclination'] < -0.6):
                node_attr[n]['pol_inclination_grp'] = -2
            elif((node_attr[n]['pol_inclination'] >= -0.6) and ((node_attr[n]['pol_inclination'] < -0.2))):
                node_attr[n]['pol_inclination_grp'] = -1
            elif((node_attr[n]['pol_inclination'] >= -0.2) and ((node_attr[n]['pol_inclination'] <= 0.2))):
                node_attr[n]['pol_inclination_grp'] = 0
            elif((node_attr[n]['pol_inclination'] > 0.2) and ((node_attr[n]['pol_inclination'] <= 0.6))):
                node_attr[n]['pol_inclination_grp'] = 1
            elif(node_attr[n]['pol_inclination'] > 0.6):
                node_attr[n]['pol_inclination_grp'] = 2
                
        nx.set_node_attributes(temp_G, node_attr)
        hom = nx.attribute_assortativity_coefficient(temp_G, "pol_inclination_grp")
        net_homophily.append(hom)

        for n in temp_G.nodes:
            node_attr[n]['pol_inclination_grp'] = round(node_attr[n]['pol_inclination'] * 10)        
        nx.set_node_attributes(temp_G, node_attr)
        
        hom = nx.attribute_assortativity_coefficient(temp_G, "pol_inclination_grp")
        
        if(math.isnan(hom) and len(set(nx.get_node_attributes(temp_G, "pol_inclination_grp").values())) == 1):
            hom = 1
        
        polarization.append(pol)
        network_homophily.append(hom)
        polarity.append(avg_pol)
        polarization2.append(Data_obj.compute_polarization(pol_inclination))

#         agent_states_df = pd.DataFrame(states)
#         agent_pol_inclination_df = pd.DataFrame(agent_pol_inclination)
#         user_activity_df = pd.DataFrame(users_activities)

        agent_states_df = pd.DataFrame()
        agent_pol_inclination_df = pd.DataFrame()
        user_activity_df = pd.DataFrame()
    
    return updated_data, G_agents, sharing_details, polarization, polarization2, network_homophily, net_homophily, polarity, user_satisfaction, user_activity, agent_states_df, agent_pol_inclination_df, user_activity_df
    
def save_results_to_dir(run, epoch, data, mypath, sharing_details, net_polarization, net_polarization2, network_homophily, net_homophily, polarity, user_satisfaction, user_activity, agent_states_df, agent_pol_inclination_df, user_activity_df):
    
    results_df = pd.DataFrame(sharing_details, columns = ['party_mentioned', 'post_stance', 'author_id', 'num_of_agents_received', 'num_of_agents_not_received', 
                                                                  'num_of_spreader_agents', 'num_of_disinterested_agents'])
    
    results_df['network_polarization'] = net_polarization
    results_df['net_polarization2'] = net_polarization2
    results_df['network_homophily'] = network_homophily
    results_df['network_homophily2'] = net_homophily
    results_df['network_polarity'] = polarity
    results_df['user_satisfaction'] = user_satisfaction
#     results_df['user_activity'] = user_activity
    
    #mypath = '../results/sharing_details/'
    results_df.to_csv(mypath + 'results_' + str(run) + '.csv')
#     agent_states_df.to_csv(mypath + 'agent_states_' + str(run) + '.csv')
#     agent_pol_inclination_df.to_csv(mypath + 'agent_polIncl_' + str(run) + '.csv')
#     user_activity_df.to_csv(mypath + 'user_activity_' + str(run) + '.csv')
    #data.to_csv(mypath + 'network_data_' + str(epoch) + '.csv')
    
    return

def save_graph(run, i, polarization, flag):

    fig, ax = plt.subplots()
    plt.plot(range(len(polarization)), polarization)
    if(flag == 3):
        filepath = '../results/results_' + str(run) + '/user_satisfaction_' + str(i) +'.jpg'
    elif(flag == 2):
        filepath = '../results/results_' + str(run) + '/network_polarity_' + str(i) +'.jpg'
    elif(flag == 1):
        filepath = '../results/results_' + str(run) + '/polarization_' + str(i) +'.jpg'
    elif(flag == 0):
        filepath = '../results/results_' + str(run) + '/network_homophily_' + str(i) +'.jpg'
        
    plt.savefig(filepath)
    plt.show()
    
    return
    
def save_data(run, i, data, flag, mypath):
    
    if os.path.isdir(mypath) == False:
        os.mkdir(mypath)
        
    if(flag == 1):
        data.to_csv(mypath + 'initial_data.csv')
    elif(flag == 2):
        data.to_csv(mypath + 'final_data_' + str(run) + str(i) +'.csv')
    
    return

In [4]:
# Data = Data()

In [5]:
# initial_data = pd.read_csv('data/initial_data_3.csv')
# initial_data_2 = pd.read_csv('../results/sharing_details_1/initial_data.csv')

In [6]:
# initial_data.pol_inclination.mean()

In [7]:
# n_issues= 6
post_conf =  pd.read_csv('data/post_conf.csv')
# initial_graph = Data.get_fb_network(initial_data)

In [8]:
# post_conf = post_conf.sample(frac = 0.002)

### Selective Exposure

In [ ]:
sjt_flag= True
lower_attd_th = 0.6
upper_attd_th = 1.4
se_flags = [False, True, True, True]
se_thresholds = [0, 0.4, 1.0, 1.6]
n_issues = 6
issue_weights = [1] * n_issues
# post_conf =  pd.read_csv('data/posts_conf.csv')
# seeds = [x for x in range(3,11)]
seeds = [15]

for k in seeds:
    
    random.seed(k)
    data_path = 'data/initial_data_' + str(k) + '.csv'
    initial_data = pd.read_csv(data_path)
#     print(initial_data.pol_inclination.mean())
    # initial_data_2 = pd.read_csv('../results/sharing_details_1/initial_data.csv')
    Data_obj = Data(k)
    initial_graph = Data_obj.get_fb_network(initial_data)

    for i in range(4):
        
        if((k == 3) and (i == 0)):
            continue

        se_threshold = se_thresholds[i]
        se_flag = se_flags[i]

        data = initial_data.copy()
        G = initial_graph.copy()

        run = str(k) + str(i)
        #print(i, end = "\t")
        #run = i

        rep = abs(data[data['pol_inclination'] < 0]['pol_inclination'].sum())
        dem = abs(data[data['pol_inclination'] > 0]['pol_inclination'].sum())
        initial_pol = round((rep + dem)/data.shape[0], 6) 
        initial_homophily = nx.attribute_assortativity_coefficient(G, "pol_inclination")
        
        if(math.isnan(initial_homophily) and len(set(nx.get_node_attributes(G, "pol_inclination").values())) == 1):
            initial_homophily = [1]
        
        mypath = 'results/new_results/SE/'
#         save_data(run, 0, initial_data, 1, mypath)
    #     for i in range(2):

        data, G, sharing_details, net_polarization, net_polarization2, network_homophily, net_homophily, polarity, user_satisfaction, user_activity, agent_states_df, agent_pol_inclination_df, user_activity_df = start_simulation(i, k, data.copy(), G, post_conf, n_issues, lower_attd_th, upper_attd_th, k, Data_obj)
        # net_polarization = initial_pol + net_polarization
        # network_homophily = initial_homophily + network_homophily
        save_results_to_dir(run, i, data.copy(), mypath, sharing_details, net_polarization, net_polarization2, network_homophily, net_homophily, polarity, user_satisfaction, user_activity, agent_states_df, agent_pol_inclination_df, user_activity_df)
#         save_graph(run, i, net_polarization, 1)
#         save_graph(run, i, network_homophily, 0)
#         save_graph(run, i, polarity, 2)
#         save_graph(run, i, user_satisfaction, 3)
        save_data(run, i, data.copy(), 2, mypath)

15 	 0 	


In [11]:
x

NameError: name 'x' is not defined